# JEM092 Asset Pricing
# Seminar 10
## Lukáš Petrásek
### Charles University
### lukas.petrasek@fsv.cuni.cz
## 21.4.2022

In [ ]:
# Import third-party packages.
library(lubridate)
library(quantmod)
library(sandwich)
library(xts)

# Data preparation

In [ ]:
load('Asset_Pricing_seminar_data.RData')
ls()

In [ ]:
START_DATE <- '2005-01-01'
END_DATE <- '2021-03-31'

# Load stock specific data.
prices <- c()
market_caps <- c()
for (i in 1:length(OHLCV_sap100)) {
    ticker <- as.character(names(OHLCV_sap100)[i])
    stock_data <- getSymbols(
        ticker,
        from = START_DATE,
        to = END_DATE,
        src = 'yahoo',
        warnings = FALSE,
        auto.assign = FALSE
    )

    stock_prices = stock_data[, 6]
    stock_market_caps <- MktCap_sap100[[i]]
    colnames(stock_prices) <- ticker
    colnames(stock_market_caps) <- ticker
    prices <- cbind(prices, stock_prices)
    market_caps <- cbind(market_caps, stock_market_caps)
}

# Load daily FF3 factors.
download.file(
    "http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_daily_TXT.zip",
    destfile = "F-F_Research_Data_Factors_daily.zip"
)
unzip("F-F_Research_Data_Factors_daily.zip")
ff3_factors <- read.delim(
    'F-F_Research_Data_Factors_daily.txt',
    col.names = c('t', 'mkt_rf', 'smb', 'hml', 'rf'),
    sep = '',
    nrows = 24957,
    header = FALSE,
    skip = 5,
    stringsAsFactors = FALSE
)
ff3_factors[['t']] <- as.Date(as.character(ff3_factors[['t']]), '%Y%m%d')
ff3_factors <- as.xts(ff3_factors[, 2:5], order.by = ff3_factors[['t']])
ff3_factors <- ff3_factors / 100

# Load monthly FF3 factors.
download.file(
    "http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_TXT.zip",
    destfile = "F-F_Research_Data_Factors.zip"
)
unzip("F-F_Research_Data_Factors.zip")
monthly_ff3_factors <- read.delim(
    'F-F_Research_Data_Factors.txt',
    col.names = c('t', 'mkt.rf', 'smb', 'hml', 'rf'),
    sep = '',
    nrows = 1137,
    header = FALSE,
    skip = 4,
    stringsAsFactors = FALSE
)
monthly_ff3_factors[['t']] <- as.Date(paste0(as.character(monthly_ff3_factors[['t']]), '01'), '%Y%m%d') # 202001
monthly_ff3_factors <- as.xts(monthly_ff3_factors[, 2:5], order.by = monthly_ff3_factors[['t']])
monthly_ff3_factors <- monthly_ff3_factors / 100
index(monthly_ff3_factors) <- as.Date(as.yearmon(index(monthly_ff3_factors)), frac = 1)

In [ ]:
prices[1:5, 1:3]
market_caps[201:205, 1:3]
ff3_factors[1:5,]
monthly_ff3_factors[1:5,]

In [ ]:
# Compute daily and monthly excess returns.
daily_returns <- lapply(prices, dailyReturn, USE.NAMES = TRUE)
monthly_returns <- lapply(prices, monthlyReturn, USE.NAMES = TRUE)
daily_returns <- do.call('cbind', daily_returns)
monthly_returns <- do.call('cbind', monthly_returns)
colnames(daily_returns) <- colnames(prices)
colnames(monthly_returns) <- colnames(prices)
index(monthly_returns) <- as.Date(as.yearmon(index(monthly_returns)), frac = 1)
for (i in 1:ncol(monthly_returns)) {
    daily_returns[, i] <- daily_returns[, i] - ff3_factors[, 4]
    monthly_returns[, i] <- monthly_returns[, i] - monthly_ff3_factors[, 4]
}

# Comupte monthly market caps and sizes.
sizes <- log(market_caps)
monthly_market_caps <- apply.monthly(market_caps, tail, 1)
monthly_sizes <- apply.monthly(sizes, tail, 1)
index(monthly_market_caps) <- as.Date(as.yearmon(index(monthly_market_caps)), frac = 1)
index(monthly_sizes) <- as.Date(as.yearmon(index(monthly_sizes)), frac = 1)
# Replace -Inf and Inf in `monthly_sizes` with NAs.
monthly_sizes <- as.xts(apply(monthly_sizes, 2, function(x) ifelse(is.finite(x), x, NA)))

In [ ]:
daily_returns[1:5, 1:3]
monthly_returns[1:5, 1:3]
monthly_market_caps[1:5, 1:3]
monthly_sizes[1:5, 1:3]

## Estimate betas

Use the following regression equation:

$r_{i,t} = \alpha_i + \beta_i MKT_t + \epsilon_{i,t}$

- $i$ denotes stocks
- $t$ denotes time
- $r$ is the excess stock return
- $MKT$ is the excess market return
- $\alpha$ is the intercept
- $\beta$ is the slope coefficient
- $\epsilon$ is the error term

Let's illustrate the estimation on daily data for 5 different periods: 1, 3, 6, 12, and 24 months.

In [ ]:
# Initialize an xts object for storing betas.
betas_period <- as.xts(
    matrix(
        nrow = nrow(monthly_returns),
        ncol = ncol(monthly_returns)
    ),
    order.by = index(monthly_returns)
)
names(betas_period) <- names(monthly_returns)
betas <- list(betas_period, betas_period, betas_period, betas_period, betas_period)

# Iterate over rows and compute betas.
for (i in 1:nrow(betas_period)) {
    end_day <- index(betas_period[i])

    for (p_b in list(c(1, 1), c(3, 2), c(6, 3), c(12, 4), c(24, 5))) {
        start_day <- end_day %m-% months(p_b[[1]])
        dates <- paste0(start_day, '/', end_day)
        returns <- daily_returns[dates]
        market_returns <- ff3_factors[dates]

        # Estimate betas for each stock.
        for (j in 1:ncol(betas_period)) {
            stock_returns <- returns[, j]
            # Merge excess returns and market excess returns.
            model_data <- cbind(stock_returns, market_returns[, 1])
            model_data <- na.omit(model_data)
            # Don't estimate betas if the number of observations is lower than 200.
            if (nrow(model_data) < p_b[[1]] * 15) {
                next
            }
            model_betas <- lm(model_data[, 1] ~ model_data[, 2])
            # Save the given beta to `betas`.
            betas[[p_b[[2]]]][i, j] <- model_betas$coefficients[[2]]
        }
    }
}

monthly_betas_1m <- betas[[1]]
monthly_betas_3m <- betas[[2]]
monthly_betas_6m <- betas[[3]]
monthly_betas_12m <- betas[[4]]
monthly_betas_24m <- betas[[5]]

In [ ]:
monthly_betas_1m[1:5, 1:3]
monthly_betas_3m[1:5, 1:3]
monthly_betas_6m[11:15, 1:3]
monthly_betas_12m[21:25, 1:3]
monthly_betas_24m[21:25, 1:3]

# Beta

## Portfolio analysis using monthly univariate decile portfolios

### Equal-weighted average returns

In [ ]:
n_portfolios <- 10
diff_portfolio <- paste(n_portfolios, '- 1')

# Initialize an xts object for storing portfolio returns.
returns_period <- as.xts(
    matrix(
        nrow = nrow(betas_period) - 1,
        ncol = n_portfolios + 1
    ),
    order.by = index(betas_period[2:nrow(betas_period)])
)
names(returns_period) <- c(1:n_portfolios, diff_portfolio)
returns_betas <- list(returns_period, returns_period, returns_period, returns_period, returns_period)

# Iterate over rows, find breakpoints and compute monthly returns within the given beta breakpoints.
for (i in 1:nrow(returns_period)) {
    current_month <- index(returns_period[i])
    next_month <- as.Date(as.yearmon(current_month %m+% months(1)), frac = 1)

    if (!next_month %in% index(monthly_returns)) {
        next
    }

    # Avoid look-ahead bias by sorting the returns after the month used to compute breakpoints ends.
    month_returns <- monthly_returns[next_month]
    for (period in 1:5) {
        month_betas <- betas[[period]][current_month]

        breakpoints <- quantile(month_betas, 0:n_portfolios/n_portfolios, na.rm = TRUE)
        not_na <- !is.na(month_betas)

        for (j in 1:n_portfolios) {
            filter <- (breakpoints[[j]] < month_betas) & (month_betas < breakpoints[[j + 1]]) & not_na
            # Compute equal-weighted average portfolio returns.
            returns_betas[[period]][i, j] <- mean(month_returns[, filter])
        }
        returns_betas[[period]][i, diff_portfolio] <- returns_betas[[period]][i, n_portfolios] - returns_betas[[period]][i, 1]
    }
}

# Compute overall average returns within portfolios and their standard errors.
results_period <- as.data.frame(matrix(nrow = 2, ncol = n_portfolios + 1))
names(results_period) <- c(1:n_portfolios, diff_portfolio)
results_betas <- list(results_period, results_period, results_period, results_period, results_period)

for (period in 1:5) {
    for (i in 1:ncol(results_betas[[period]])) {
        model <- lm(na.omit(returns_betas[[period]][, i]) ~ 1)
        results_betas[[period]][1, i] <- model$coefficients[[1]]
        results_betas[[period]][2, i] <- model$coefficients[[1]] / sqrt(NeweyWest(model, lag = 6))[[1]]
    }
}

# Results ordered in the following way: 1, 3, 6, 12, and 24 months.
results_betas

### Market-capitalization-weighted returns

In [ ]:
n_portfolios <- 10
diff_portfolio <- paste(n_portfolios, '- 1')

# Initialize an xts object for storing portfolio returns.
returns_period <- as.xts(
    matrix(
        nrow = nrow(betas_period) - 1,
        ncol = n_portfolios + 1
    ),
    order.by = index(betas_period[2:nrow(betas_period)])
)
names(returns_period) <- c(1:n_portfolios, diff_portfolio)
returns_betas <- list(returns_period, returns_period, returns_period, returns_period, returns_period)

# Iterate over rows, find breakpoints and compute monthly returns within the given beta breakpoints.
for (i in 1:nrow(returns_period)) {
    current_month <- index(returns_period[i])
    next_month <- as.Date(as.yearmon(current_month %m+% months(1)), frac = 1)

    if (!next_month %in% index(monthly_returns)) {
        next
    }

    # Avoid look-ahead bias by sorting the returns after the month used to compute breakpoints ends.
    month_returns <- monthly_returns[next_month]
    month_market_caps <- monthly_market_caps[next_month]
    # Replace NA market caps with 0s so that such observations have no weights.
    month_market_caps[is.na(month_market_caps)] <- 0
    for (period in 1:5) {
        month_betas <- betas[[period]][current_month]

        breakpoints <- quantile(month_betas, 0:n_portfolios/n_portfolios, na.rm = TRUE)
        not_na <- !is.na(month_betas)

        for (j in 1:n_portfolios) {
            filter <- (breakpoints[[j]] < month_betas) & (month_betas < breakpoints[[j + 1]]) & not_na
            # Compute weighted average portfolio returns.
            returns_betas[[period]][i, j] <- weighted.mean(t(month_returns[, filter]), t(month_market_caps[, filter]))
        }
        returns_betas[[period]][i, diff_portfolio] <- returns_betas[[period]][i, n_portfolios] - returns_betas[[period]][i, 1]
    }
}

# Compute overall average returns within portfolios and their standard errors.
results_period <- as.data.frame(matrix(nrow = 2, ncol = n_portfolios + 1))
names(results_period) <- c(1:n_portfolios, diff_portfolio)
results_betas <- list(results_period, results_period, results_period, results_period, results_period)

for (period in 1:5) {
    for (i in 1:ncol(results_betas[[period]])) {
        model <- lm(na.omit(returns_betas[[period]][, i]) ~ 1)
        results_betas[[period]][1, i] <- model$coefficients[[1]]
        results_betas[[period]][2, i] <- model$coefficients[[1]] / sqrt(NeweyWest(model, lag = 6))[[1]]
    }
}

# Results ordered in the following way: 1, 3, 6, 12, and 24 months.
results_betas

## Fama-MacBeth regression analysis

In [ ]:
# Initialize an xts object for storing regression results.
fm_cs_results_period <- as.xts(
    matrix(
        nrow = nrow(betas_period) - 1,
        ncol = 5
    ),
    order.by = index(betas_period[2:nrow(betas_period)])
)
names(fm_cs_results_period) <- c('Intercept', 'Beta', 'R^2', 'Adjusted R^2', 'N')
fm_cs_results <- rep(list(fm_cs_results_period), 5)

# Iterate over rows and perform cross-sectional regressions.
for (i in 1:nrow(fm_cs_results_period)) {
    current_month <- index(fm_cs_results_period[i])
    next_month <- as.Date(as.yearmon(current_month %m+% months(1)), frac = 1)

    if (!next_month %in% index(monthly_returns)) {
        next
    }

    # Avoid look-ahead bias by sorting the returns after the quarter used to compute breakpoints ends.
    month_returns <- monthly_returns[next_month]

    for (period in 1:5) {
        month_betas <- betas[[period]][current_month]

        if (ncol(month_betas) == sum(is.na(month_betas))) {
            next
        }

        # Save regression coefficients and other statistics.
        model <- lm(t(month_returns) ~ 1 + t(month_betas))
        fm_cs_results[[period]][i, 1] <- model$coefficients[[1]]
        fm_cs_results[[period]][i, 2] <- model$coefficients[[2]]
        fm_cs_results[[period]][i, 3] <- summary(model)$r.squared
        fm_cs_results[[period]][i, 4] <- summary(model)$adj.r.squared
        fm_cs_results[[period]][i, 5] <- nobs(model)
    }
}

# Compute time series means, standard errors.
fm_results_period <- as.data.frame(matrix(nrow = 2, ncol = 2))
names(fm_results_period) <- c('Intercept', 'Beta')
fm_results_betas <- rep(list(fm_results_period), 5)

for (period in 1:5) {
    for (i in 1:ncol(fm_results_period)) {
        model <- lm(na.omit(fm_cs_results[[period]][, i]) ~ 1)
        fm_results_betas[[period]][1, i] <- model$coefficients[[1]]
        fm_results_betas[[period]][2, i] <- model$coefficients[[1]] / sqrt(NeweyWest(model, lag = 4))[[1]]
    }
}

# Results ordered in the following way: 1, 3, 6, 12, and 24 months.
fm_results_betas

# Size

## Portfolio analysis using monthly univariate decile portfolios

### Equal-weighted average returns

In [ ]:
n_portfolios <- 10
diff_portfolio <- paste(n_portfolios, '- 1')

# Initialize an xts object for storing portfolio returns.
returns_sizes <- as.xts(
    matrix(
        nrow = nrow(monthly_sizes) - 1,
        ncol = n_portfolios + 1
    ),
    order.by = index(monthly_sizes[2:nrow(monthly_sizes)])
)
names(returns_sizes) <- c(1:n_portfolios, diff_portfolio)

# Iterate over rows, find breakpoints and compute monthly returns within the given size breakpoints.
for (i in 1:nrow(returns_sizes)) {
    current_month <- index(returns_sizes[i])
    next_month <- as.Date(as.yearmon(current_month %m+% months(1)), frac = 1)

    if (!next_month %in% index(monthly_returns)) {
        next
    }

    # Avoid look-ahead bias by sorting the returns after the month used to compute breakpoints ends.
    month_returns <- monthly_returns[next_month]
    month_sizes <- monthly_sizes[current_month]

    breakpoints <- quantile(month_sizes, 0:n_portfolios/n_portfolios, na.rm = TRUE)
    not_na <- !is.na(month_sizes)

    for (j in 1:n_portfolios) {
        filter <- (breakpoints[[j]] < month_sizes) & (month_sizes < breakpoints[[j + 1]]) & not_na
        # Compute equal-weighted average portfolio returns.
        returns_sizes[i, j] <- mean(month_returns[, filter])
    }
    returns_sizes[i, diff_portfolio] <- returns_sizes[i, n_portfolios] - returns_sizes[i, 1]
}

# Compute overall average returns within portfolios and their standard errors.
results_sizes <- as.data.frame(matrix(nrow = 2, ncol = n_portfolios + 1))
names(results_sizes) <- c(1:n_portfolios, diff_portfolio)

for (i in 1:ncol(results_sizes)) {
    model <- lm(na.omit(returns_sizes[, i]) ~ 1)
    results_sizes[1, i] <- model$coefficients[[1]]
    results_sizes[2, i] <- model$coefficients[[1]] / sqrt(NeweyWest(model, lag = 6))[[1]]
}

results_sizes

### Market-capitalization-weighted returns

In [ ]:
n_portfolios <- 10
diff_portfolio <- paste(n_portfolios, '- 1')

# Initialize an xts object for storing portfolio returns.
returns_sizes <- as.xts(
    matrix(
        nrow = nrow(monthly_sizes) - 1,
        ncol = n_portfolios + 1
    ),
    order.by = index(monthly_sizes[2:nrow(monthly_sizes)])
)
names(returns_sizes) <- c(1:n_portfolios, diff_portfolio)

# Iterate over rows, find breakpoints and compute monthly returns within the given size breakpoints.
for (i in 1:nrow(returns_sizes)) {
    current_month <- index(returns_sizes[i])
    next_month <- as.Date(as.yearmon(current_month %m+% months(1)), frac = 1)

    if (!next_month %in% index(monthly_returns)) {
        next
    }

    # Avoid look-ahead bias by sorting the returns after the month used to compute breakpoints ends.
    month_returns <- monthly_returns[next_month]
    month_sizes <- monthly_sizes[current_month]
    month_market_caps <- monthly_market_caps[next_month]
    # Replace NA market caps with 0s so that such observations have no weights.
    month_market_caps[is.na(month_market_caps)] <- 0

    breakpoints <- quantile(month_sizes, 0:n_portfolios/n_portfolios, na.rm = TRUE)
    not_na <- !is.na(month_sizes)

    for (j in 1:n_portfolios) {
        filter <- (breakpoints[[j]] < month_sizes) & (month_sizes < breakpoints[[j + 1]]) & not_na
        # Compute equal-weighted average portfolio returns.
        returns_sizes[i, j] <- weighted.mean(t(month_returns[, filter]), t(month_market_caps[, filter]))
    }
    returns_sizes[i, diff_portfolio] <- returns_sizes[i, n_portfolios] - returns_sizes[i, 1]
}

# Compute overall average returns within portfolios and their standard errors.
results_sizes <- as.data.frame(matrix(nrow = 2, ncol = n_portfolios + 1))
names(results_sizes) <- c(1:n_portfolios, diff_portfolio)

for (i in 1:ncol(results_sizes)) {
    model <- lm(na.omit(returns_sizes[, i]) ~ 1)
    results_sizes[1, i] <- model$coefficients[[1]]
    results_sizes[2, i] <- model$coefficients[[1]] / sqrt(NeweyWest(model, lag = 6))[[1]]
}

results_sizes

## Fama-MacBeth regression analysis

In [ ]:
# Initialize an xts object for storing regression results.
fm_cs_results <- as.xts(
    matrix(
        nrow = nrow(monthly_sizes) - 1,
        ncol = 5
    ),
    order.by = index(monthly_sizes[2:nrow(monthly_sizes)])
)
names(fm_cs_results) <- c('Intercept', 'Size', 'R^2', 'Adjusted R^2', 'N')

# Iterate over rows and perform cross-sectional regressions.
for (i in 1:nrow(fm_cs_results)) {
    current_month <- index(fm_cs_results[i])
    next_month <- as.Date(as.yearmon(current_month %m+% months(1)), frac = 1)

    if (!next_month %in% index(monthly_returns)) {
        next
    }

    # Avoid look-ahead bias by sorting the returns after the quarter used to compute breakpoints ends.
    month_returns <- monthly_returns[next_month]
    month_sizes <- monthly_sizes[current_month]

    # Save regression coefficients and other statistics.
    model <- lm(t(month_returns) ~ 1 + t(month_sizes))
    fm_cs_results[i, 1] <- model$coefficients[[1]]
    fm_cs_results[i, 2] <- model$coefficients[[2]]
    fm_cs_results[i, 3] <- summary(model)$r.squared
    fm_cs_results[i, 4] <- summary(model)$adj.r.squared
    fm_cs_results[i, 5] <- nobs(model)
}

# Compute time series means, standard errors.
fm_results_sizes <- as.data.frame(matrix(nrow = 2, ncol = 2))
names(fm_results_sizes) <- c('Intercept', 'Size')

for (i in 1:ncol(fm_results_sizes)) {
    model <- lm(na.omit(fm_cs_results[, i]) ~ 1)
    fm_results_sizes[1, i] <- model$coefficients[[1]]
    fm_results_sizes[2, i] <- model$coefficients[[1]] / sqrt(NeweyWest(model, lag = 4))[[1]]
}

fm_results_sizes

In [ ]:
tail(monthly_sizes[, 11:15])
# Market caps denominated in USD billions.
tail(monthly_market_caps[, 11:15])
exp(1)

## Portfolio analysis using monthly portfolios formed by bivariate dependent sorting

In [ ]:
n_portfolios_beta <- 3
n_portfolios_size <- 4
n_portfolios <- n_portfolios_beta * n_portfolios_size
diff_portfolios <- c('4 - 1', '8 - 5', '12 - 9')

# Initialize an xts object for storing portfolio returns.
returns_sorts <- as.xts(
    matrix(
        nrow = nrow(monthly_betas_24m) - 1,
        ncol = n_portfolios + n_portfolios_beta
    ),
    order.by = index(monthly_betas_24m[2:nrow(monthly_betas_24m)])
)
names(returns_sorts) <- c(1:n_portfolios, diff_portfolios)

# Iterate over rows, find breakpoints and compute monthly returns within the given beta and size breakpoints.
for (i in 1:nrow(returns_sorts)) {
    current_month <- index(returns_sorts[i])
    next_month <- as.Date(as.yearmon(current_month %m+% months(1)), frac = 1)

    if (!next_month %in% index(monthly_returns)) {
        next
    }

    # Avoid look-ahead bias by sorting the returns after the month used to compute breakpoints ends.
    month_returns <- monthly_returns[next_month]
    month_betas <- monthly_betas_24m[current_month]
    month_sizes <- monthly_sizes[current_month]
    month_market_caps <- monthly_market_caps[next_month]
    # Replace NA market caps with 0s so that such observations have no weights.
    month_market_caps[is.na(month_market_caps)] <- 0

    if (ncol(month_betas) == sum(is.na(month_betas))) {
        next
    }

    breakpoints_betas <- quantile(month_betas, 0:n_portfolios_beta/n_portfolios_beta, na.rm = TRUE)
    not_na_betas <- !is.na(month_betas)

    for (j in 1:n_portfolios_beta) {
        filter_betas <- (breakpoints_betas[[j]] < month_betas) & (month_betas < breakpoints_betas[[j + 1]]) & not_na_betas
        filtered_month_returns <- month_returns[, filter_betas]
        filtered_month_sizes <- month_sizes[, filter_betas]
        filtered_month_market_caps <- month_market_caps[, filter_betas]
        breakpoints_sizes <- quantile(filtered_month_sizes, 0:n_portfolios_size/n_portfolios_size, na.rm = TRUE)
        not_na_sizes <- !is.na(filtered_month_sizes)
        for (k in 1:n_portfolios_size) {
            filter_sizes <- (breakpoints_sizes[[k]] < filtered_month_sizes) & (filtered_month_sizes < breakpoints_sizes[[k + 1]]) & not_na_sizes
            # Compute weighted average portfolio returns.
            returns_sorts[i, k + (j - 1) * n_portfolios_size] <- weighted.mean(t(filtered_month_returns[, filter_sizes]), t(filtered_month_market_caps[, filter_sizes]))
        }
        returns_sorts[i, diff_portfolios[j]] <- returns_sorts[i, n_portfolios_size * j] - returns_sorts[i, n_portfolios_size * (j - 1) + 1]
    }
}

# Compute overall average returns within portfolios and their standard errors.
results_sizes <- as.data.frame(matrix(nrow = 2, ncol = n_portfolios + n_portfolios_beta))
names(results_sizes) <- c(1:n_portfolios, diff_portfolios)

for (i in 1:ncol(results_sizes)) {
    model <- lm(na.omit(returns_sorts[, i]) ~ 1)
    results_sizes[1, i] <- model$coefficients[[1]]
    results_sizes[2, i] <- model$coefficients[[1]] / sqrt(NeweyWest(model, lag = 6))[[1]]
}
results_sizes

# Fama-MacBeth regressions of returns on betas and sizes

In [ ]:
# Initialize an xts object for storing regression results.
fm_cs_results <- as.xts(
    matrix(
        nrow = nrow(monthly_betas_24m) - 1,
        ncol = 6
    ),
    order.by = index(monthly_betas_24m[2:nrow(monthly_betas_24m)])
)
names(fm_cs_results) <- c('Intercept', 'Beta', 'Size', 'R^2', 'Adjusted R^2', 'N')

# Iterate over rows and perform cross-sectional regressions.
for (i in 1:nrow(fm_cs_results)) {
    current_month <- index(fm_cs_results[i])
    next_month <- as.Date(as.yearmon(current_month %m+% months(1)), frac = 1)

    if (!next_month %in% index(monthly_returns)) {
        next
    }

    # Avoid look-ahead bias by sorting the returns after the quarter used to compute breakpoints ends.
    month_returns <- monthly_returns[next_month]
    month_betas <- monthly_betas_24m[current_month]
    month_sizes <- monthly_sizes[current_month]

    if (ncol(month_betas) == sum(is.na(month_betas))) {
        next
    }

    # Save regression coefficients and other statistics.
    model <- lm(t(month_returns) ~ 1 + t(month_betas) + t(month_sizes))
    fm_cs_results[i, 1] <- model$coefficients[[1]]
    fm_cs_results[i, 2] <- model$coefficients[[2]]
    fm_cs_results[i, 3] <- model$coefficients[[3]]
    fm_cs_results[i, 4] <- summary(model)$r.squared
    fm_cs_results[i, 5] <- summary(model)$adj.r.squared
    fm_cs_results[i, 6] <- nobs(model)
}

# Compute time series means, standard errors.
fm_results_sizes <- as.data.frame(matrix(nrow = 2, ncol = 3))
names(fm_results_sizes) <- c('Intercept', 'Beta', 'Size')

for (i in 1:ncol(fm_results_sizes)) {
    model <- lm(na.omit(fm_cs_results[, i]) ~ 1)
    fm_results_sizes[1, i] <- model$coefficients[[1]]
    fm_results_sizes[2, i] <- model$coefficients[[1]] / sqrt(NeweyWest(model, lag = 4))[[1]]
}

fm_results_sizes